# Azure AI Agents - File Search

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [1]:
import os
import sys

from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv
from openai import AzureOpenAI
from azure.ai.projects.models import FileSearchTool, MessageAttachment, FilePurpose

In [2]:
load_dotenv("azure.env")

True

In [3]:
sys.version

'3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]'

## Project

In [4]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.getenv("PROJECT_CONNECTION_STRING"),
)

In [5]:
model="gpt-4o"

In [6]:
DATA_DIR = "data"

os.makedirs(DATA_DIR, exist_ok=True)

output_file = os.path.join(DATA_DIR, "document.pdf")

In [7]:
!wget https://arxiv.org/abs/2311.06242 -O $output_file

--2025-05-20 13:03:28--  https://arxiv.org/abs/2311.06242
Resolving arxiv.org (arxiv.org)... 151.101.131.42, 151.101.3.42, 151.101.195.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.131.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49384 (48K) [text/html]
Saving to: ‘data/document.pdf’

data/document.pdf   100%[===================>]  48.23K  --.-KB/s    in 0.002s  

2025-05-20 13:03:28 (29.0 MB/s) - ‘data/document.pdf’ saved [49384/49384]



In [8]:
file = project_client.agents.upload_file_and_poll(file_path=output_file,
                                                  purpose=FilePurpose.AGENTS)

print(f"Uploaded file, file ID: {file.id}")

# create a vector store with the file you uploaded
vector_store = project_client.agents.create_vector_store_and_poll(
    file_ids=[file.id], name="document_vector_store")

print(f"Created vector store, vector store ID: {vector_store.id}")

Uploaded file, file ID: assistant-5iXPwrHG7pQFQrM4x4cvUW
Created vector store, vector store ID: vs_fdQ9fF7pJ5C4ZaAZGxH0jzMt


In [9]:
# create a file search tool
file_search_tool = FileSearchTool(vector_store_ids=[vector_store.id])

# notices that FileSearchTool as tool and tool_resources must be added or the agent will be unable to search the file
agent = project_client.agents.create_agent(
    model=model,
    name="document_agent",
    instructions="You are an AI helpful agent to analyse document",
    tools=file_search_tool.definitions,
    tool_resources=file_search_tool.resources,
)

print(f"Created agent, agent ID: {agent.id}")

Created agent, agent ID: asst_UzWDa2x48dwZN2nNql1SK7xw


In [10]:
# Create a thread
thread = project_client.agents.create_thread()
print(f"Created thread, thread ID: {thread.id}")

# Upload the user provided file as a messsage attachment
message_file = project_client.agents.upload_file_and_poll(
    file_path=output_file, purpose=FilePurpose.AGENTS)

print(f"Uploaded file, file ID: {message_file.id}")

# Create a message with the file search attachment
# Notice that vector store is created temporarily when using attachments with a default expiration policy of seven days.
attachment = MessageAttachment(file_id=message_file.id,
                               tools=FileSearchTool().definitions)

prompt = "Summarize this document in one line"

message = project_client.agents.create_message(thread_id=thread.id,
                                               role="user",
                                               content=prompt,
                                               attachments=[attachment])

print(f"Created message, message ID: {message.id}")

Created thread, thread ID: thread_FMHCvnSCvGNrqQoEsi6XZfIm
Uploaded file, file ID: assistant-LT8pwLhPdXkUEseRzq96it
Created message, message ID: msg_fEziO4c9RC5xn10jAryeeukM


In [11]:
run = project_client.agents.create_and_process_run(thread_id=thread.id,
                                                   agent_id=agent.id)
print(f"Created run, run ID: {run.id}")

messages = project_client.agents.list_messages(thread_id=thread.id)
print(f"Messages: {messages}")

Created run, run ID: run_WfNzF2cAOSu2VGCnxQDcPJVc
Messages: {'object': 'list', 'data': [{'id': 'msg_TbHq5cZ6jmcUkGXhWfXPJ0uI', 'object': 'thread.message', 'created_at': 1747746227, 'assistant_id': 'asst_UzWDa2x48dwZN2nNql1SK7xw', 'thread_id': 'thread_FMHCvnSCvGNrqQoEsi6XZfIm', 'run_id': 'run_WfNzF2cAOSu2VGCnxQDcPJVc', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'The document introduces Florence-2, a groundbreaking vision foundation model employing a unified, prompt-based approach to excel across diverse computer vision and vision-language tasks with outstanding zero-shot and fine-tuning capabilities【4:0†source】.', 'annotations': [{'type': 'file_citation', 'text': '【4:0†source】', 'start_index': 241, 'end_index': 253, 'file_citation': {'file_id': 'assistant-LT8pwLhPdXkUEseRzq96it'}}]}}], 'attachments': [], 'metadata': {}}, {'id': 'msg_fEziO4c9RC5xn10jAryeeukM', 'object': 'thread.message', 'created_at': 1747746217, 'assistant_id': None, 'thread_id': 'thread_FMHCvnS

In [12]:
print(messages.data[0].content[0].text.value)

The document introduces Florence-2, a groundbreaking vision foundation model employing a unified, prompt-based approach to excel across diverse computer vision and vision-language tasks with outstanding zero-shot and fine-tuning capabilities【4:0†source】.


## Another question

In [13]:
prompt = "What is FLD-5B?"

message = project_client.agents.create_message(thread_id=thread.id,
                                               role="user",
                                               content=prompt,
                                               attachments=[attachment])

print(f"Created message, message ID: {message.id}")

Created message, message ID: msg_4hxgJcdm14qSJvGyO04Uq1fy


In [14]:
run = project_client.agents.create_and_process_run(thread_id=thread.id,
                                                   agent_id=agent.id)

print(f"Created run, run ID: {run.id}")

messages = project_client.agents.list_messages(thread_id=thread.id)

Created run, run ID: run_QIcR16I2EGWNK1nszOSj3oHX


In [15]:
print(messages.data[0].content[0].text.value)

FLD-5B is a large-scale dataset comprising 5.4 billion visual annotations spread across 126 million images, created by an iterative process of automated annotation and model refinement, and primarily developed to train the Florence-2 model for versatile vision tasks【8:0†source】.


## Post processing

In [16]:
agents = project_client.agents.list_agents()

for i in range(len(agents.data)):
    print(agents.data[i])
    print()

{'id': 'asst_XVqmSXT6vuAmwJHuhDbFXLEm', 'object': 'assistant', 'created_at': 1747746226, 'name': 'Agent with code interpreter', 'description': None, 'model': 'gpt-4o-mini', 'instructions': 'You are a helpful AI agent that can analyse input file for statistics', 'tools': [{'type': 'code_interpreter'}], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {'code_interpreter': {'file_ids': ['assistant-RBH6Pm5L773FFcZjSLTvcq']}}, 'metadata': {}, 'response_format': 'auto'}

{'id': 'asst_UzWDa2x48dwZN2nNql1SK7xw', 'object': 'assistant', 'created_at': 1747746214, 'name': 'document_agent', 'description': None, 'model': 'gpt-4o', 'instructions': 'You are an AI helpful agent to analyse document', 'tools': [{'type': 'file_search'}], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {'file_search': {'vector_store_ids': ['vs_fdQ9fF7pJ5C4ZaAZGxH0jzMt']}}, 'metadata': {}, 'response_format': 'auto'}



In [17]:
project_client.agents.delete_vector_store(vector_store.id)
print("Deleted vector store")

project_client.agents.delete_agent(agent.id)
print("Deleted agent")

Deleted vector store
Deleted agent


In [18]:
# Delete the original file from the agent to free up space
print("Deleted file")
project_client.agents.delete_file(file.id)
print("Done")

Deleted file
Done
